# Temporary Spectrum Sensing Algorithm

In [1]:
import numpy as np
import random
from FPU import cmul, cadd
from utils import conj, read_binary, binary_to_fp16, store_binary, generate_fp16, fp16_to_binary, int_to_binary, binary_to_int
from ed import ed

In [9]:
# -----------------------------------------------------------------------------
# Helper: generate a single frame and compute its energy via ed
# -----------------------------------------------------------------------------
def make_frame(num_samples=128, seed=None):
    """
    Create one complex FP16 frame with `num_samples` using generate_fp16.
    """
    if seed is not None:
        # Keep Python's random for generate_fp16 consistency
        pass
    frame = generate_fp16(size=num_samples, complex=True, seed=seed)
    return frame

def frame_energy(frame):
    """
    Compute energy of a frame using ed.
    Returns a Python float (from FP16 real accumulator).
    """

    acc, _ = ed(frame)
    # Energy is the real part; imag should be ~0
    return float(np.float16(acc[0]))

# -----------------------------------------------------------------------------
# Main flow
# -----------------------------------------------------------------------------
def main():
    # 1) Generate 10 frames, each with 128 complex FP16 samples.
    #    Use different seeds to ensure different frames.
    frames = []
    for i in range(10):
        frames.append(make_frame(num_samples=128, seed=42 + i))

    # 2) Compute energy per frame using ed
    energies = []
    for idx, fr in enumerate(frames):
        e = frame_energy(fr)  # set False to see ed's prints
        energies.append((idx, e))  # keep original index to map back later

    # 3) Sort energies ascending and build thresholds between specific order stats.
    #    We want counts:
    #      - 6 frames: E < th1
    #      - 1 frame:  th1 < E < th4
    #      - 1 frame:  th4 < E < th3
    #      - 1 frame:  th3 < E < th2
    #      - 1 frame:  E > th2
    #
    #    Achieve this by placing thresholds between consecutive sorted energies:
    #      th1 between E[5] and E[6]
    #      th4 between E[6] and E[7]
    #      th3 between E[7] and E[8]
    #      th2 between E[8] and E[9]
    #
    #    Using midpoints guarantees strict inequalities and no ties.
    energies_sorted = sorted(energies, key=lambda x: x[1])
    E = [e for (_, e) in energies_sorted]

    # Safety: in rare cases energies can tie; perturb mid by tiny epsilon if needed.
    def midpoint(a, b):
        mid = 0.5 * (a + b)
        # If a == b, nudge upward to enforce strict ordering
        if a == b:
            mid = a + 1e-6
        return mid

    th1 = midpoint(E[5], E[6])
    th4 = midpoint(E[6], E[7])
    th3 = midpoint(E[7], E[8])
    th2 = midpoint(E[8], E[9])

    # 4) Verify the bucket counts exactly match the specification.
    def bucket(e):
        if e < th1:
            return "below_th1"
        elif th1 < e < th4:
            return "th1_th4"
        elif th4 < e < th3:
            return "th4_th3"
        elif th3 < e < th2:
            return "th3_th2"
        elif e > th2:
            return "above_th2"
        else:
            # If e equals some threshold due to rare FP16 collisions,
            # nudge comparisons by tiny eps.
            eps = 1e-9
            if e <= th1 + eps:
                return "below_th1"
            if e >= th2 - eps:
                return "above_th2"
            # For the middle two, resolve ties by ordering to lower bucket
            if e <= th4 + eps:
                return "th1_th4"
            if e <= th3 + eps:
                return "th4_th3"
            return "th3_th2"

    counts = {"below_th1": 0, "th1_th4": 0, "th4_th3": 0, "th3_th2": 0, "above_th2": 0}
    assignment = []
    for idx, e in energies:
        b = bucket(e)
        counts[b] += 1
        assignment.append((idx, e, b))

    # 5) Print results
    print("=== Frame Energies (unsorted) ===")
    for idx, e in enumerate([e for (_, e) in energies]):
        print(f"Frame {energies[idx][0]:2d} : Energy = {energies[idx][1]:.6f}")

    print("\n=== Sorted Energies ===")
    for rank, (idx, e) in enumerate(energies_sorted):
        print(f"Rank {rank:2d} | Frame {idx:2d} : Energy = {e:.6f}")

    print("\n=== Thresholds ===")
    print(f"th1 = {th1:.6f}")
    print(f"th4 = {th4:.6f}")
    print(f"th3 = {th3:.6f}")
    print(f"th2 = {th2:.6f}")

    print("\n=== Bucket Counts (should be 6,1,1,1,1) ===")
    print(counts)

    print("\n=== Frame → Bucket Assignment ===")
    for idx, e, b in sorted(assignment, key=lambda x: x[0]):
        print(f"Frame {idx:2d} : Energy = {e:.6f} -> {b}")

    print("\n=== Frame → Bucket Assignment (by Sorted Energy) ===")
    for idx, e in energies_sorted:
        # assignment에서 해당 frame의 bucket을 찾아오기
        b = next(b for (i, ee, b) in assignment if i == idx)
        print(f"Frame {idx:2d} : Energy = {e:.6f} -> {b}")

    frames_sorted = [frames[idx] for (idx, _) in energies_sorted]

    return frames, frames_sorted


In [10]:
if __name__ == "__main__":
    # You can switch to use the original ed (with prints) by changing
    frames, frames_sorted = main()

=== Frame Energies (unsorted) ===
Frame  0 : Energy = 259.250000
Frame  1 : Energy = 258.500000
Frame  2 : Energy = 244.000000
Frame  3 : Energy = 273.500000
Frame  4 : Energy = 258.000000
Frame  5 : Energy = 257.250000
Frame  6 : Energy = 263.750000
Frame  7 : Energy = 248.000000
Frame  8 : Energy = 242.875000
Frame  9 : Energy = 281.750000

=== Sorted Energies ===
Rank  0 | Frame  8 : Energy = 242.875000
Rank  1 | Frame  2 : Energy = 244.000000
Rank  2 | Frame  7 : Energy = 248.000000
Rank  3 | Frame  5 : Energy = 257.250000
Rank  4 | Frame  4 : Energy = 258.000000
Rank  5 | Frame  1 : Energy = 258.500000
Rank  6 | Frame  0 : Energy = 259.250000
Rank  7 | Frame  6 : Energy = 263.750000
Rank  8 | Frame  3 : Energy = 273.500000
Rank  9 | Frame  9 : Energy = 281.750000

=== Thresholds ===
th1 = 258.875000
th4 = 261.500000
th3 = 268.625000
th2 = 277.625000

=== Bucket Counts (should be 6,1,1,1,1) ===
{'below_th1': 6, 'th1_th4': 1, 'th4_th3': 1, 'th3_th2': 1, 'above_th2': 1}

=== Frame → 

In [ ]:
for `i in range(10):
    store_binary(frames[i], f'../PE/DATA/ss_{i}_input.txt', zero_padding=0)

In [11]:
for i in range(10):
    store_binary(frames_sorted[i], f'../PE/DATA/ss_sorted_{i}_input.txt', zero_padding=0)

In [ ]:
a = fp16_to_binary(258.875000)
print(a)
b = binary_to_int(a)
print(b)
c = int_to_binary(b)
print(c)

a = fp16_to_binary(261.500000)
print(a)
b = binary_to_int(a)
print(b)
c = int_to_binary(b)
print(c)

a = fp16_to_binary(268.625000)
print(a)
b = binary_to_int(a)
print(b)
c = int_to_binary(b)
print(c)

a = fp16_to_binary(277.625000)
print(a)
b = binary_to_int(a)
print(b)
c = int_to_binary(b)
print(c)

0101110000001100
23564
0101110000001100
0101110000010110
23574
0101110000010110
0101110000110010
23602
0101110000110010
0101110001010110
23638
0101110001010110
